Transformar as tabelas local.csv e saida_series_temporais.xlsx em uma tabela com a estrutura: {Nome da PEV, Latitude, Longitude, [Distancia para todas as outras Pevs], Media de produção}


In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

Transformando os registros por mes em médias de rendimento 

In [2]:
def diff_month(d1, d2):
    diff = (d1.year - d2.year) * 12 + d1.month - d2.month
    if(diff != 0): return diff
    else: return 1 

In [57]:
df = pd.read_csv("saida_series_temporais - SemiTratado.csv", delimiter=",")

newDf = pd.DataFrame(columns=["PEV", "MediaRendimento"])

for i in range (1,195):
    registrosPev = df.loc[df["LOCAL"] == i]
    
    maiordata = 0; menordata = 0; soma = 0; media = 0
    for index,row in registrosPev.iterrows():
        soma += float(row["VALOR"])
        d = row["DATA"]
        dataAtual = datetime(int(d[0:4]),int(d[4:6]),int(d[6:8]),int(d[9:11]),int(d[12:14]),int(d[15:17])) # converte a data em str pra datetime
        if(maiordata == 0 and menordata == 0):
            maiordata = dataAtual
            menordata = dataAtual
        if(menordata > dataAtual):
            menordata = dataAtual
        elif(maiordata < dataAtual):
            maiordata = dataAtual    
        if(len(registrosPev)>0):      
            media = soma / diff_month(maiordata,menordata)
    newDf.loc[len(newDf)] = [i,media]

newDf.to_csv("mediaDeRendimentoPEVs.csv", index=False)


Criando a tabela final

In [4]:
from math import radians, sin, cos, sqrt, atan2

In [5]:
def calculaDistancia(lat1,long1,lat2,long2):
    R = 6371  # Raio da Terra em km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, long1, lat2, long2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c # Retorna distancia em km

In [11]:
dfLocal = pd.read_csv("local.csv", delimiter=";")
dfMedias = pd.read_csv("mediaDeRendimentoPEVs.csv", delimiter=",")

dist_cols = [f"DistanciaPEV{i+1}" for i in range(len(dfLocal))]
newDf = pd.DataFrame(columns=["PEV", "latitude", "longitude"] + dist_cols + ["MediaRendimento"])

# Remover espaços extras e linhas vazias em "PEV"
dfLocal = dfLocal[dfLocal["PEV"].notna()]
dfLocal["PEV"] = dfLocal["PEV"].str.strip().str.replace(r"\s+", " ", regex=True)

# Extrair o número do PEV como inteiro
dfLocal["PEV_NUM"] = dfLocal["PEV"].str.extract(r"(\d+)").astype(int)


# Lista de colunas de distância
dist_cols = [f"DistanciaPEV{num}" for num in dfLocal["PEV_NUM"]]
newDf = pd.DataFrame(columns=["PEV", "latitude", "longitude"] + dist_cols + ["MediaRendimento"])

# Preenchendo o novo DataFrame
for i, row in dfLocal.iterrows():
    cols = [row["PEV_NUM"], row["latitude"], row["longitude"]]

    # Calcular distâncias para todos os outros PEVs
    for _, row2 in dfLocal.iterrows():
        dist = calculaDistancia(row["latitude"], row["longitude"], row2["latitude"], row2["longitude"])
        cols.append(dist)

    media_row = dfMedias.loc[dfMedias["PEV"] == row["PEV_NUM"]]
    media = round(media_row["Media de Rendimento"].values[0],2) if not media_row.empty else None
    cols.append(media)

    if(media != 0):
        newDf.loc[len(newDf)] = cols

newDf.to_csv("DistanciaEMediasPevs.csv", index=False)
    

In [6]:
dfLocal = pd.read_csv("local.csv", delimiter=";")

dfLocal = dfLocal[dfLocal["PEV"].notna()]
# Extrair o número do PEV como inteiro
dfLocal["PEV_NUM"] = dfLocal["PEV"].str.extract(r"(\d+)").astype(int)

dfLocal = dfLocal.set_index("PEV_NUM")

dfLocal = dfLocal[["latitude", "longitude"]]

In [7]:
dfLocal

,latitude,longitude
PEV_NUM,,
1,-27.701280,-49.334159
2,-27.697261,-49.333901
3,-27.507223,-49.286285
4,-27.422946,-48.957696
5,-27.904127,-49.128131
...,...,...
190,-27.491927,-48.725521
191,-27.499486,-48.662853
192,-27.277885,-48.852346


In [8]:
dfMedias = pd.read_csv("mediaDeRendimentoPEVs.csv", delimiter=",", index_col="PEV")

newDf = dfMedias.join(dfLocal)

newDf = newDf[newDf.MediaRendimento != 0]

newDf.to_csv("localERendimento.csv")
